In [8]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [9]:
from pathlib import Path
from datetime import datetime as dt
import pandas as pd

In [ ]:
path="/content/drive/MyDrive/ARNER/ANERCorp.xlsx"
xlsx = pd.ExcelFile(path)
df=pd.read_excel(xlsx, header=None)
df

In [ ]:
df = df.drop(2, 1)
df = df.rename(columns={0: 'text', 1: 'label'})
df

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)
train_arr = []
test_arr = []
train_lbl = []
test_lbl = []

In [14]:
train_arr=train['text'].astype(str)
train_lbl=train['label'].astype(str)
test_arr=test['text'].astype(str)
test_lbl=test['label'].astype(str)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)
test_mat = vectorizer.transform(test_arr)
test_tfmat = tfidf.transform(test_mat)

In [16]:
del df
del test_arr
del train_arr

In [18]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.svm import LinearSVC
train_tfmat

lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [19]:
y_pred_lsvm=lsvm.predict(test_tfmat)

In [20]:
test=['ألمانيا']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'B-LOC'

In [21]:

from sklearn.metrics import  accuracy_score
from sklearn import metrics
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

accuracy: 0.9375207931332756


In [43]:
import pickle
# save the model to disk
modelfilename = '/content/drive/MyDrive/ARNER/lsvm.sav'
pickle.dump(lsvm, open(modelfilename, 'wb'))

#save vectorizer
vectorizerfilename = '/content/drive/MyDrive/ARNER/vectorizer.pickle'
pickle.dump(vectorizer, open(vectorizerfilename, 'wb'))

#save tfidf
tfidffilename = '/content/drive/MyDrive/ARNER/tfidf.pickle'
pickle.dump(tfidf, open(tfidffilename, 'wb'))

In [46]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
import pickle
import pandas as pd
# load the model from disk
modelfilename='/content/drive/MyDrive/ARNER/lsvm.sav'
loaded_model = pickle.load(open(modelfilename, 'rb'))

# load the vectorizer from disk
vectorizerfilename='/content/drive/MyDrive/ARNER/vectorizer.pickle'
loaded_vectorizer = pickle.load(open(vectorizerfilename, 'rb'))


# load the tfidf from disk
tfidffilename='/content/drive/MyDrive/ARNER/tfidf.pickle'
loaded_tfidf = pickle.load(open(tfidffilename, 'rb'))


In [47]:
phrase="هو مؤسس التقويم الهجري، وفي عهده بلغ الإسلام مبلغًا عظيمًا، وتوسع نطاق الدولة الإسلامية حتى شمل كامل العراق ومصر وليبيا والشام وفارس وخراسان وشرق الأناضول وجنوب أرمينية وسجستان،"

In [48]:
import sys
arr=phrase.split()
y=[]
token=[]
for x in arr:
    x=[x]
    test_str = loaded_vectorizer.transform(x)
    test_tfstr = loaded_tfidf.transform(test_str)
    test_tfstr.shape
    token.append(x)
    y.append(loaded_model.predict(test_tfstr.toarray())[0])

df=pd.DataFrame(list(zip(token,y)),columns=['token','entity_type'])
df

,token,entity_type
0,[هو],O
1,[مؤسس],O
2,[التقويم],O
3,[الهجري،],O
4,[وفي],O
5,[عهده],O
6,[بلغ],O
7,[الإسلام],B-MISC
8,[مبلغًا],O
9,[عظيمًا،],O
